<div style="width: 30%; float: right; margin: 10px; margin-right: 5%;">
    <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d3/FHNW_Logo.svg/2560px-FHNW_Logo.svg.png" width="500" style="float: left; filter: invert(50%);"/>
</div>

# Phi-2 Few-Shot learning

In diesem Notebook werden wir einen Chatbot für Schweizer Immobilien Empfehlungen mittels Few-Shot learning erstellen. <br>
Dabei verwenden wir das LLM [phi-2](https://huggingface.co/microsoft/phi-2 ) von Microsoft.



---
Bearbeitet durch Si Ben Tran, Yannic Lais, Rami Tarabishi im HS 2023.<br>
Bachelor of Science FHNW in Data Science.

## Einleitung

### Allgemeines Vorgehen

- Name entity recognition auf den Prompt
- Entities werden für die Datenbankabfrage extrahiert
- Prompt wird mit den Trainingsexamples sowie der Datenbankabfrage an das Phi-2 Modell gesendet

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import re
from torchsummary import summary

c:\Users\yanni\miniconda3\envs\npr_mc2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

NVIDIA GeForce RTX 4060 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


## Immobiliendaten

In [3]:
# read parquet file
df = pd.read_parquet('data\immo_data_202208.parquet')

In [4]:
# filter for important columns
df = df[['Municipality', 'detailed_description', 'price_cleaned', 'type', 'rooms', 'url']]
df['price'] = df['price_cleaned'].astype(float)
df['rooms'] = pd.to_numeric(df['rooms'], errors='coerce')
df = df.drop(columns=['price_cleaned'])
# drop rows with missing values
df = df.dropna()

In [5]:
df.head()

,Municipality,detailed_description,type,rooms,url,price
0,Biberstein,DescriptionLuxuriöse Attika-Wohnung direkt an ...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,1150000.0
1,Biberstein,DescriptionStilvolle Liegenschaft an ruhiger L...,terrace-house,5.0,https://www.immoscout24.ch//en/d/terrace-house...,1420000.0
3,Biberstein,DescriptionDieses äusserst grosszügige Minergi...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,1430000.0
4,Küttigen,DescriptionAus ehemals zwei Wohnungen wurde ei...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-romb...,995000.0
5,Erlinsbach (AG),DescriptionDer Blick in die Weite vermittelt R...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,2160000.0


In [6]:
df.dtypes

Municipality             object
detailed_description     object
type                     object
rooms                   float64
url                      object
price                   float64
dtype: object

## Phi-2 Model

In [7]:
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-2", torch_dtype="auto", device_map="cuda", trust_remote_code=True)
model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)

c:\Users\yanni\miniconda3\envs\npr_mc2\lib\site-packages\torch\autocast_mode.py:156: UserWarning: In CPU autocast, but the target dtype is not supported. Disabling autocast.
CPU Autocast only supports dtype of torch.bfloat16 currently.
  warnings.warn(error_message)
Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.10s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
'''input_size = (10, 512)
summary(model, input_size=input_size)'''

'input_size = (10, 512)\nsummary(model, input_size=input_size)'

#### Einzelner Input

In [9]:
inputs = tokenizer("Is a penguin a bird or a mamal?", return_tensors="pt").to('cuda')

# Generate outputs and decode
outputs = model.generate(**inputs, max_length=40)
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text)

Is a penguin a bird or a mamal?
Answer: A penguin is a bird.

Exercise 2:
What is the difference between a simile and a metaphor


## Verschiedene Queries

### Funktionen für Query processing

In [10]:
# Few-shot examples
few_shot_examples = [
    {
        "Question": "I am looking for an flat in Zurich under 1'000'000 CHF.", 
        "Answer": "Here are some options for apartments in Zurich under 1'000'000 CHF: (max_price = 1000000, location_keyword = Zürich, property_type = flat)"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "Yes, there are terraced houses in Bern in the CHF 500,000 to 700,000 range: (max_price = 700000, min_price = 500000, location_keyword = Bern, property_type = terraced_house)"
    },
    {
        "Question": "I need a detached house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "In Lucerne you can find detached houses with a garden for around CHF 1,200,000: (location_keyword = Bern, property_type = detached-house, arround_price = 1200000)"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "Modern apartments in Basel under 900'000 CHF are available: (max_price = 900000, location_keyword = Basel, property_type = flat, rooms = 3.5)"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "Large houses in Lausanne with at least 5 rooms up to 1'500'000 CHF can be found here: (max_price = 1500000, location_keyword = Lausanne, property_type = house)"
    }

]


In [11]:
def filter_dataframe(df = df, max_price = None, min_price = None, arround_price = None, location_keyword = None, property_type = None, rooms = None):

    type_list =  df.type.unique().tolist()
    type_list.append('house')
    type_list = [x.lower() for x in type_list]

    # Apply filters
    if arround_price:
        filtered_df = df[df['price'] <= arround_price * 1.1]
        filtered_df = df[df['price'] >= arround_price * 0.9]
    if max_price:
        filtered_df = df[df['price'] <= max_price]
    if min_price:
        filtered_df = df[df['price'] >= min_price]
    if location_keyword:
        filtered_df = filtered_df[filtered_df['Municipality'].str.contains(location_keyword, case=False, na=False)]
    if property_type.lower() in type_list:
        filtered_df = filtered_df[filtered_df['type'].str.contains(property_type, case=False, na=False)]
    if rooms:
        filtered_df = filtered_df[filtered_df['rooms'] == rooms]

    # Return 5 random samples
    if len(filtered_df) >= 5:
        return filtered_df.sample(n=5, random_state = 42)
    else:
        return filtered_df

In [12]:
test = filter_dataframe(df, max_price = 1200000, location_keyword = 'Basel', property_type = 'flat')

In [13]:
test.head()

,Municipality,detailed_description,type,rooms,url,price
2742,Basel,DescriptionObjektbeschrieb An idealer Lage im...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,495000.0
2738,Basel,DescriptionDiese Wohnung im 3. Obergeschoss bi...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,650000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2737,Basel,DescriptionIm Breite-Quartier wird diese heime...,flat,2.0,https://www.immoscout24.ch//en/d/flat-buy-base...,470000.0
2733,Basel,DescriptionBaujahr: 1961Renovation: 2022Wohnfl...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,695000.0


In [14]:
def get_model_response(query, model, tokenizer, examples = few_shot_examples):
    # Load model and tokenizer
    model = model
    tokenizer = tokenizer

    # Format the input with few-shot examples
    prompt_text = "\n\n".join([f"Question: {ex['Question']}\nAnswer: {ex['Answer']}" for ex in examples])
    prompt_text += f"\n\nQuestion: {query}"

    # Encode and send to model
    inputs = tokenizer(prompt_text, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs, max_length=600, num_return_sequences=1)

    # Decode the output
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extracting the answer corresponding to the specific query
    response_parts = full_response.split("Answer:")
    for i, part in enumerate(response_parts[:-1]):
        if f"Question: {query}" in part:
            return response_parts[i + 1].split("\n")[0].strip()

    return "No specific answer found."

In [15]:
def parse_and_filter(input_str, df):
    try:
        # Extract the parameter string
        params_str = re.search(r'\((.*?)\)', input_str).group(1)
    
    except:
        return "No specific answer found."

    # Initialize parameters with default values
    params = {
        'df': df,
        'max_price': None,
        'min_price': None,
        'arround_price': None,
        'location_keyword': None,
        'property_type': None,
        'rooms': None
    }

    # Split the parameter string and iterate over each parameter
    for param in params_str.split(','):
        key, value = param.split('=')
        key = key.strip()
        value = value.strip()

        # Convert value to the correct type if necessary
        if key in ['max_price', 'min_price', 'arround_price', 'rooms']:
            value = int(value)
        # Update the parameters dictionary
        params[key] = value

    # Call the filter_dataframe function with unpacked arguments
    return filter_dataframe(**params)

### Query 1: Intent wie in Beispielen

In [16]:
# Example usage
query = "Show me real estate in Zurich which cost less than 4'000'000 CHF."

response = get_model_response(query, model=model, tokenizer=tokenizer)
filtered_df = parse_and_filter(response, df)

print(response.split(':')[0])
filtered_df

Here are some options for real estate in Zurich under 4'000'000 CHF


,Municipality,detailed_description,type,rooms,url,price
13152,"Kilchberg (ZH), Kreis 2 (Zurich)","DescriptionSouverän und symmetrisch, kraftvoll...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-kilc...,2570000.0
13157,"Kilchberg (ZH), Kreis 2 (Zurich)","DescriptionSouverän und symmetrisch, kraftvoll...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-kilc...,3050000.0
13266,"Dübendorf, Kreis 7 (Zurich)","Description4,5-Zimmer-Wohnung mit 2 Parkplätze...",flat,4.0,https://www.immoscout24.ch//en/d/flat-buy-gock...,1530000.0


#### Evaluation

In [17]:
# show how many Municiplaities with the keyword 'Basel' are in the dataset
df[df['Municipality'].str.contains('Genf', case=False, na=False)].count()

Municipality            0
detailed_description    0
type                    0
rooms                   0
url                     0
price                   0
dtype: int64

### Query 2: in Deutsch

In [18]:
query_german = "Zeige mir ein paar Wohnungen in Basel, die weniger als 1'200'000 CHF kosten."

response = get_model_response(query_german, model=model, tokenizer=tokenizer)
filtered_df = parse_and_filter(response, df)

print(response.split(':')[0])
filtered_df

Here are a few apartments in Basel that cost less than 1'200'000 CHF


,Municipality,detailed_description,type,rooms,url,price
2737,Basel,DescriptionIm Breite-Quartier wird diese heime...,flat,2.0,https://www.immoscout24.ch//en/d/flat-buy-base...,470000.0
2742,Basel,DescriptionObjektbeschrieb An idealer Lage im...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,495000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2786,Basel,DescriptionDiese Wohnung überzeugt insbesonder...,duplex-maisonette,5.0,https://www.immoscout24.ch//en/d/duplex-maison...,875000.0
2732,Basel,DescriptionAn sehr schöner und attraktiver Lag...,penthouse,5.0,https://www.immoscout24.ch//en/d/penthouse-buy...,950000.0


#### Evaluation

Wohnungen werden automatisch als Apartments erkannt, es gäbe weitere Wohnungen mit dem porperty_type: flat, apartment etc.

In [19]:
query_german_df = filter_dataframe(df, max_price = 1200000, location_keyword = 'Basel', property_type = 'Flat')
query_german_df.head()

,Municipality,detailed_description,type,rooms,url,price
2742,Basel,DescriptionObjektbeschrieb An idealer Lage im...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,495000.0
2738,Basel,DescriptionDiese Wohnung im 3. Obergeschoss bi...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,650000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2737,Basel,DescriptionIm Breite-Quartier wird diese heime...,flat,2.0,https://www.immoscout24.ch//en/d/flat-buy-base...,470000.0
2733,Basel,DescriptionBaujahr: 1961Renovation: 2022Wohnfl...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,695000.0


### Query 3: mit anderem Intent

In [20]:
query_random = "Hello do you have bread?"

response = get_model_response(query_random, model=model, tokenizer=tokenizer)
filtered_df = parse_and_filter(response, df)

print(response.split(':')[0])
filtered_df

Yes, we have bread.


'No specific answer found.'

#### Evaluation

Das Phi-2 Modell erkennt korrekt, wenn sich die Query auf einen anderen Intent bezieht und antwortet darauf. <br>
Zudem wiedergibt die Immobilienabfrage korrekt, dass keine 

### Query 4: extrahieren des gesammten Dataframe Abfrage Codes

In [27]:
few_shot_examples_code = [
    {
        "Question": "I am looking for an flat in Zurich under 1'000'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('flat'))]"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]"
    },
    {
        "Question": "I need a house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment'))]"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Lausanne')) & (df['rooms'] >= 5) & (df['price'] <= 1500000) & (df['type'].str.contains('house'))]"
    }
]


In [28]:
query = "Show me flats in Basel which cost less than 2'000'000 CHF with 3 or more rooms."

response = get_model_response(query, model=model, tokenizer=tokenizer, examples=few_shot_examples_code)

print(response)

exec(response)
df_test.head()

df_test = df[(df['Municipality'].str.contains('Basel')) & (df['price'] < 2000000) & (df['rooms'] >= 3) & (df['type'].str.contains('flat'))]


,Municipality,detailed_description,type,rooms,url,price
2670,Basel,"DescriptionGrosszügige, sonnige 3.5-Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1450000.0
2671,Basel,DescriptionWollen Sie sich endlich Ihren Traum...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1130000.0
2675,Basel,"DescriptionDie helle, charmante 4.5 Zimmerwohn...",flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1280000.0
2705,Basel,DescriptionDie moderne 3-Zimmer Wohnung befind...,flat,3.0,https://www.immoscout24.ch//en/d/flat-buy-base...,670000.0
2709,Basel,DescriptionDiese helle und geräumige Etagenwoh...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-base...,1380000.0


In [31]:
query = "I am looking for a house in Biberstein which costs about 1'000'000."

response = get_model_response(query, model=model, tokenizer=tokenizer, examples=few_shot_examples_code)

print(response)
    
exec(response)
df_test.head()

df_test = df[(df['Municipality'].str.contains('Biberstein')) & (df['price'] == 1000000) & (df['type'].str.contains('house'))]


,Municipality,detailed_description,type,rooms,url,price


Hier wird keine Preisspanne abgefragt, sondern nur der exakte Preis abgefragt.

In [32]:
query = "I need a house in Basel with a garden for around CHF 1,200,000."

response = get_model_response(query, model=model, tokenizer=tokenizer, examples=few_shot_examples_code)

print(response)

exec(response)
df_test.head()

df_test = df[(df['Municipality'].str.contains('Basel')) & (df['price'] <= (1200000*1.1)) & (df['price'] >= (1200000*0.9)) & (df['type'].str.contains('house'))]


,Municipality,detailed_description,type,rooms,url,price
2701,Basel,DescriptionNur wenige Schritte vom Rhein entfe...,terrace-house,7.0,https://www.immoscout24.ch//en/d/terrace-house...,1290000.0
2715,Basel,DescriptionAn einer ruhigen Nebenstrasse in de...,detached-house,5.0,https://www.immoscout24.ch//en/d/detached-hous...,1250000.0


Wählen wir die Query fast identisch wie ein Example wird der Filter korrekt verwendet.

#### Evaluation

Der Code wird erstaunlich gut generiert.

### Query 5: Test test mit multiplen typen

In [38]:
few_shot_examples_multiple_type_code = [
    {
        "Question": "I am looking for an flat in Zurich under 1'000'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Zürich')) & (df['price'] < 1000000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]"
    },
    {
        "Question": "Are there terraced houses in Bern in the CHF 500,000 to 700,000 range?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] >= 500000) & (df['price'] <= 700000) & (df['type'].str.contains('terraced_house'))]"
    },
    {
        "Question": "I need a detached house in Lucerne with a garden for around CHF 1,200,000.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Luzern')) & (df['price'] == 1200000) & (df['type'].str.contains('house'))]"
    },
    {
        "Question": "Are there modern apartments with 3.5 rooms available in Basel for under CHF 900,000?",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Basel')) & (df['rooms'] == 3.5) & (df['price'] < 900000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]"
    },
    {
        "Question": "I am looking for a large house in Lausanne, at least 5 rooms, up to 1'500'000 CHF.",
        "Answer": "df_test = df[(df['Municipality'].str.contains('Lausanne')) & (df['rooms'] >= 5) & (df['price'] <= 1500000) & (df['type'].str.contains('house'))]"
    }
]

In [44]:
query = "Show me apartments in Bern for less than 1'000'000 CHF."

response = get_model_response(query, model=model, tokenizer=tokenizer, examples=few_shot_examples_multiple_type_code)

print(response)

exec(response)
df_test.head()

df_test = df[(df['Municipality'].str.contains('Bern')) & (df['price'] < 1000000) & (df['type'].str.contains('apartment|penthouse|flat|attic-room'))]


,Municipality,detailed_description,type,rooms,url,price
1728,Bern,DescriptionIdéalement situé dans le quartier d...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,680000.0
1735,Bern,DescriptionZentrumsnahes Wohnen in der Stadt B...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,555000.0
1741,Bern,DescriptionSchöne sonnige Wohnung auf 3.en Eta...,flat,5.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,351250.0
1745,Bern,DescriptionEin Herz für das Ostring-Quartier!A...,flat,7.0,https://www.immoscout24.ch//en/d/flat-buy-bern...,985000.0
1750,Bern,DescriptionLageDas Objekt befindet sich in ein...,attic-flat,5.0,https://www.immoscout24.ch//en/d/attic-flat-bu...,930000.0


Auch bei dem Keyword apartment werden nach allen möglichen Wohnungstypen gesucht.

#### Evaluation

## Fazit

Erstaunlich gute Performance, überrascht wie einfach die implementierung und auch die Abfragegeschwindigkeit ist

### Ausblick

- Viel mehr Datawrangling
- Einbindung in RASA chatbot

## Erkentnisse

- Max length des outputs am anfang ein problem anschliessend nicht mehr beim Fewshot example